In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

import glob
import torch 

from transformers4rec import torch as tr
from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
from transformers4rec.torch.utils.examples_utils import wipe_memory
from transformers4rec.torch.utils.data_utils import T4RecDataLoader

KeyboardInterrupt: 

In [2]:
import nvtabular as nvt
import cudf
import torch.utils.dlpack as dlpack
import tqdm

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/home/ec2-user/SageMaker/token_recommender/data/")


In [4]:
model_path= os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/saved_model")

model = tr.Model().load(model_path)

In [6]:
workflow = nvt.Workflow.load('../preprocessWorkflow')

In [8]:
S3_PATH = f"s3a://zarklab-token-recommender-ohio/buy-sell-updated-2018-11-02-2023-07-31/*.parquet"

#mydata = cudf.read_parquet(S3_PATH)
dataset = nvt.Dataset(S3_PATH,engine='parquet')

In [9]:

# mydata['timestamp'] = cudf.to_datetime(mydata['timestamp'])

# print("Count with  repeated interactions: {}".format(len(mydata)))

# # Sorts the dataframe by session and timestamp, to remove consecutive repetitions
# # mydata.timestamp = mydata.timestamp.astype(int)
# interactions_df = mydata.sort_values(['recipient', 'timestamp'])
# past_ids = interactions_df['buyAsset'].shift(1).fillna()
# session_past_ids = interactions_df['recipient'].shift(1).fillna()

# # Keeping only no consecutive repeated in session interactions
# interactions_df = interactions_df[~((interactions_df['recipient'] == session_past_ids) & (interactions_df['buyAsset'] == past_ids))]

# print("Count after removed in-session repeated interactions: {}".format(len(interactions_df)))

In [10]:
procesed = workflow.transform(dataset)

In [11]:
dataloader =  T4RecDataLoader.parse('merlin').from_schema(
            procesed.schema,
            procesed,
            1280,
            max_sequence_length=20,
            drop_last=False,
            shuffle=False
        )

In [ ]:
# procesed_df = procesed_df[["recipient",'buyAsset-list']]
# procesed_df.head()

In [10]:
# wipe_memory()

In [12]:
all_indices = []
K=10
softmax = torch.nn.Softmax(dim=1)
for i, data in enumerate(dataloader):
    out = model.forward(data[0],testing = False)
    prob = softmax(out)
    top5 = torch.topk(prob, K, dim=1,sorted=True)
    
    # score = top5.values
    indices = top5.indices

    all_indices.append(indices)
    if i % 100==0:
        print(i/len(dataloader))

0.0
0.016886187098953058
0.033772374197906116
0.05065856129685917
0.06754474839581223
0.08443093549476528
0.10131712259371833
0.1182033096926714
0.13508949679162446
0.1519756838905775
0.16886187098953057
0.18574805808848363
0.20263424518743667
0.21952043228638973
0.2364066193853428
0.25329280648429586
0.2701789935832489
0.28706518068220194
0.303951367781155
0.32083755488010807
0.33772374197906113
0.3546099290780142
0.37149611617696726
0.38838230327592027
0.40526849037487334
0.4221546774738264
0.43904086457277947
0.45592705167173253
0.4728132387706856
0.48969942586963866
0.5065856129685917
0.5234718000675448
0.5403579871664979
0.5572441742654508
0.5741303613644039
0.5910165484633569
0.60790273556231
0.6247889226612631
0.6416751097602161
0.6585612968591692
0.6754474839581223
0.6923336710570753
0.7092198581560284
0.7261060452549815
0.7429922323539345
0.7598784194528876
0.7767646065518405
0.7936507936507936
0.8105369807497467
0.8274231678486997
0.8443093549476528
0.8611955420466059
0.87808

In [13]:
all_indices_cat = torch.cat(all_indices)
# all_score_cat = torch.cat(all_score)

print(all_indices_cat.shape)
# print(all_score_cat.shape)


torch.Size([7579051, 10])


In [14]:
column_names = [f'top{x}' for x in range(1,11)]
all_indices_cat_numpy = all_indices_cat.cpu().numpy()
# Create cuDF DataFrame


In [18]:
procesed_df = procesed.to_ddf()[["recipient",'buyAsset-list']]

In [19]:
procesed_cudf = procesed_df.compute()

In [20]:
procesed_cudf.shape

(7579051, 2)

In [21]:
procesed_cudf.head()

,recipient,buyAsset-list
0,0x000000000000000000000000000000000000dead,"[50, 50, 50, 50, 50, 50, 50, 50, 50, 882, 882,..."
1,0x0000000000000000000000000000000000020855,"[897, 897]"
2,0x0000000000000d9054f605ca65a2647c2b521422,"[5, 6, 5, 5, 5, 1911, 6, 629, 6]"
3,0x000000000000d34c44564053af35e4fe271d0caa,"[3, 10, 22, 3, 22, 22, 22, 22]"
4,0x000000000004d7463d0f9c77383600bc82d612f5,"[2079, 1131, 846, 109, 2005, 5, 1969, 1121, 19..."


In [22]:
for i,col in enumerate(column_names):
    procesed_cudf[col] = all_indices_cat_numpy[:, i]

In [23]:
procesed_cudf.head(20)

,recipient,buyAsset-list,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
0,0x000000000000000000000000000000000000dead,"[50, 50, 50, 50, 50, 50, 50, 50, 50, 882, 882,...",3,13,17,56,129,154,16,11,620,693
1,0x0000000000000000000000000000000000020855,"[897, 897]",3,4,6,88,183,90,199,37,8,397
2,0x0000000000000d9054f605ca65a2647c2b521422,"[5, 6, 5, 5, 5, 1911, 6, 629, 6]",5,3,4,6,9,12,31,7,10,14
3,0x000000000000d34c44564053af35e4fe271d0caa,"[3, 10, 22, 3, 22, 22, 22, 22]",3,4,22,6,10,5,7,21,15,40
4,0x000000000004d7463d0f9c77383600bc82d612f5,"[2079, 1131, 846, 109, 2005, 5, 1969, 1121, 19...",3,908,221,542,304,6,433,341,698,675
5,0x000000000008e4db6a6194c6957df47e30970dc2,"[3, 3, 3, 3, 3, 5, 3, 5, 5, 5, 5, 3, 5, 5]",3,5,4,6,99,38,713,7,36,286
6,0x00000000000b4bdf11195d2a010c32a91b8af06a,"[5, 3, 3, 3]",5,4,6,7,10,18,38,21,22,131
7,0x00000000000b4e8b4897ef401b74ec21aaa2997a,"[5, 5]",3,4,5,6,7,810,38,713,99,31
8,0x00000000003e04625c9001717346dd811ae5eba2,"[5, 5, 5, 5, 5, 591, 5, 5, 3]",5,6,4,3,7,9,12,18,31,10
9,0x000000000057e8abd581828a2b40076852c0ba6e,"[37, 956]",3,4,6,37,10,21,90,88,8,26


In [24]:
procesed_cudf.drop(['buyAsset-list'],axis=1).to_parquet(f"s3a://zarklab-token-recommender-ohio/prediction_reuslt/2018-11-02-2023-07-31_prediction")
